In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9560,2024-05-27,Sérvia Superliga,15:00,Mega Basket,Partizan,3.37,1.29,167.5,1.85,1.85,0.0,0.00,0.00,tlHOXls3,0.296736,0.775194,0.540541,0.540541,0.071930,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,72,82,6.41,1.09,0.000,0.000,0.631237,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9561,2024-05-27,Taiwan P. League+,08:00,Hsinchu Jko Lioneers,Taoyuan Pauian Pilots,2.84,1.39,183.5,1.87,1.79,0.0,0.00,0.00,WlOeboJt,0.352113,0.719424,0.534759,0.558659,0.071537,343.230,109.529605,0.319114,1.8,1.643168,0.912871,503.86,3.512,0.763394,0.217367,2.0,139.750,25.021287,0.179043,1.8,1.643168,0.912871,122.40,1.504,0.328222,0.218233,-2.0,118,102,4.27,1.20,206.152,157.358,0.484778,0.030912,NaN,1.39,0.278,6.618705,0.487324,0.7,0.212676,0.18,0.036,10.833333,0.563139,0.6,0.036861
9562,2024-05-27,Uruguai Liga Uruguaia,21:15,Penarol,Defensor Sporting,1.39,2.84,155.5,1.82,1.88,0.0,0.00,0.00,GSOGQpJE,0.719424,0.352113,0.549451,0.531915,0.071537,112.700,13.743775,0.121950,2.4,1.341641,0.559017,130.40,1.370,0.232487,0.169698,53.0,181.688,47.412432,0.260955,1.8,1.643168,0.912871,133.59,2.282,0.694169,0.304193,24.0,80,61,1.63,2.19,119.338,231.816,0.484778,0.022933,NaN,0.07,0.014,27.857143,0.728187,0.8,0.071813,2.25,0.450,4.088889,0.411176,0.6,0.188824
9563,2024-05-27,Brasil Lbf Feminina,19:30,Campinas F,Ponta Grossa F,1.06,7.90,135.5,1.82,1.88,0.0,0.00,0.00,MTsmrCvo,0.943396,0.126582,0.549451,0.531915,0.069979,104.736,35.085449,0.334989,2.4,1.341641,0.559017,161.28,1.612,0.572818,0.355346,24.0,860.776,702.659082,0.816309,1.2,1.643168,1.369306,1825.00,13.974,12.000599,0.858781,-26.0,63,73,2.56,25.00,151.808,927.202,1.079601,0.022933,NaN,-1.03,-0.206,-0.291262,0.000000,0.0,0.000000,6.10,1.220,5.655738,0.000000,0.0,0.000000
9564,2024-05-27,Brasil Lbf Feminina,20:00,Catanduva F,Sampaio F,9.45,20.00,148.5,1.85,1.85,0.0,0.00,0.00,ENUY5FWi,0.105820,0.050000,0.540541,0.540541,-0.844180,552.968,331.012896,0.598611,0.6,1.341641,2.236068,903.00,8.428,4.815031,0.571314,-52.0,106.564,35.467955,0.332832,3.0,0.000000,0.000000,168.27,1.284,0.481643,0.375111,131.0,86,79,10.50,2.13,567.608,102.422,0.506620,0.000000,NaN,-1.02,-0.204,-41.421569,0.000000,0.0,0.000000,1.97,0.394,48.223350,0.000000,0.0,0.000000
9565,2024-05-27,Argentina Liga A,20:00,San Lorenzo,Obera TC,1.49,2.54,152.5,1.87,1.86,0.0,0.00,0.00,z9Lcxla6,0.671141,0.393701,0.534759,0.537634,0.064842,157.618,37.795352,0.239791,1.2,1.643168,1.369306,160.37,2.026,0.366988,0.181139,-6.0,142.922,30.915553,0.216311,1.2,1.643168,1.369306,162.81,1.796,0.369161,0.205546,-17.0,79,81,2.03,2.01,125.226,183.396,0.368468,0.003791,NaN,0.56,0.112,4.375000,0.605814,0.7,0.094186,-0.48,-0.096,-16.041667,0.436374,0.4,-0.036374
9566,2024-05-27,África Bal,12:00,Rivers Hoopers,Monastir,2.20,1.62,141.5,1.83,1.87,0.0,0.00,0.00,Wh2g5yKP,0.454545,0.617284,0.546448,0.534759,0.071829,100.124,18.742489,0.187193,1.2,1.643168,1.369306,99.18,1.508,0.228298,0.151391,9.0,218.668,72.892028,0.333346,2.4,1.341641,0.559017,135.80,2.996,0.916941,0.306055,12.0,57,70,1.74,1.94,0.000,0.000,0.214724,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9567,2024-05-27

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9567,12:00,Geórgia Superliga,Kavkasia,Kutaisi,2.29,Back Home
